In [1]:
import pandas as pd
from sqlalchemy.orm import Session, sessionmaker
from sqlalchemy import create_engine
from datetime import datetime
from model import Store, Item, Inventory, WastePrediction



In [2]:
data = pd.read_csv('food-waste-2025.csv', header=None, skiprows=1)
data.columns = data.iloc[0]  
data = data.drop(0).reset_index(drop=True)
data = data.dropna(how ='any')
# Ensure the timestamp column is parsed correctly
data['timestamp'] = pd.to_datetime(data['timestamp'], format='%Y-%m-%d %H:%M:%S')



In [3]:
data.head(6)

,item_id,category,store_id,timestamp,quantity,shelf_life_days,wasted,days_on_shelf,price
0,23,Dairy,5,2025-01-02 08:14:22,10,21,0,11,14.99
1,47,Prepared Foods,8,2025-01-02 11:38:45,8,2,1,1,8.99
2,12,Produce,3,2025-01-03 09:22:17,15,7,0,2,3.49
3,31,Meat,6,2025-01-03 14:57:33,7,5,0,3,15.99
4,4,Bakery,1,2025-01-04 07:45:12,6,3,1,2,5.99
5,29,Meat,7,2025-01-05 16:32:08,12,6,0,1,22.99


In [4]:
#Connect to SQLite DB using SQLAlchemy
DATABASE_URL = "sqlite:///./phood_data.db"
engine = create_engine(DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
session = SessionLocal()


In [ ]:
for index, row in data.iterrows():
    # store_id = row['store_id']
    # store = session.query(Store).filter(Store.id == store_id).first()
    # if not store:
    #     store = Store(id=store_id)
    #     session.add(store)
    store_id = row['store_id']   
    item_id = row['item_id']
    timestamp = row['timestamp']
    item = session.query(Item).filter(Item.item_id == item_id, Item.timestamp == timestamp).first()
    if not item:
        item = Item(id=item_id, category=row['category'], timestamp=timestamp)
        session.add(item)
    
        
    inventory = Inventory(
        item_id = item.id,
        store_id = store_id,
        timestamp = timestamp,
        quantity = row['quantity'],
        shelf_life_days = row['shelf_life_days'],
        wasted = row['wasted'],
        days_on_shelf = row['days_on_shelf'],
        price = row['price']
    )
    session.add(inventory)
    
    
    # Optionally insert waste prediction with a default value
    waste_prediction = WastePrediction(inventory_id=inventory.id, waste_probability=0)  # Placeholder value
    session.add(waste_prediction)
session.commit()

# Close session
session.close()


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: items.id
[SQL: INSERT INTO items (id, category) VALUES (?, ?)]
[parameters: [('23', 'Dairy'), ('47', 'Prepared Foods'), ('12', 'Produce'), ('31', 'Meat'), ('4', 'Bakery'), ('29', 'Meat'), ('18', 'Produce'), ('42', 'Prepared Foods')  ... displaying 10 of 50 total bound parameter sets ...  ('29', 'Meat'), ('37', 'Dairy')]]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [ ]:
from sqlalchemy import inspect

# Initialize the inspector object to get metadata about the database
inspector = inspect(engine)

# List all tables in the database
tables = inspector.get_table_names()
print("Tables in database:", tables)

# Iterate through all tables to print columns and foreign key relationships
for table_name in tables:
    print(f"\nTable: {table_name}")
    
    # Get the columns of the current table
    columns = inspector.get_columns(table_name)
    for column in columns:
        print(f"  Column: {column['name']} ({column['type']})")

    # Get foreign key relationships
    foreign_keys = inspector.get_foreign_keys(table_name)
    if foreign_keys:
        print("  Foreign Key Relationships:")
        for fk in foreign_keys:
            referenced_table = fk['referred_table']
            print(f"    - {table_name}.{fk['constrained_columns']} -> {referenced_table}.{fk['referred_columns']}")
    else:
        print("  No foreign key relationships.")


Tables in database: ['inventory', 'items', 'stores', 'waste_prediction']

Table: inventory
  Column: id (INTEGER)
  Column: item_id (INTEGER)
  Column: store_id (INTEGER)
  Column: timestamp (DATETIME)
  Column: quantity (INTEGER)
  Column: shelf_life_days (INTEGER)
  Column: wasted (BOOLEAN)
  Column: days_on_shelf (INTEGER)
  Column: price (FLOAT)
  Foreign Key Relationships:
    - inventory.['item_id'] -> items.['id']
    - inventory.['store_id'] -> stores.['id']

Table: items
  Column: id (INTEGER)
  Column: name (VARCHAR)
  Column: category (VARCHAR)
  No foreign key relationships.

Table: stores
  Column: id (INTEGER)
  No foreign key relationships.

Table: waste_prediction
  Column: id (INTEGER)
  Column: inventory_id (INTEGER)
  Column: waste_probability (FLOAT)
  Foreign Key Relationships:
    - waste_prediction.['inventory_id'] -> inventory.['id']
